In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import random_split, DataLoader

import torchvision
from torchvision import models, transforms
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import ImageFolder

import torchmetrics

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger

In [11]:
class DERMNETDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage):
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        test_transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        train_full = ImageFolder(self.data_dir + '\\train', train_transform)
        data_len = len(train_full)
        val_len = int(0.05*data_len)
        self.train_ds, self.val_ds = random_split(train_full, [data_len-val_len, val_len])
        self.test_ds = ImageFolder(self.data_dir + '\\test', test_transform)

    def train_dataloader(self):
        return DataLoader(self.train_ds, self.batch_size, num_workers=9)

    def val_dataloader(self):
        return DataLoader(self.val_ds, self.batch_size, num_workers=9)

In [12]:
DATA_DIR = r"C:\Users\15148\OneDrive - Vanier College\Desktop\Datasets\CancerDataset+Rashes\archive"
BATCH_SIZE = 5
NUM_CLASSES = 23

In [13]:
data_module = DERMNETDataModule(data_dir=DATA_DIR, batch_size=BATCH_SIZE)

In [14]:
class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.loss = nn.MultiLabelMarginLoss()
        self.accuracy = torchmetrics.Accuracy(average='macro', num_classes=NUM_CLASSES)
        self.F1 = torchmetrics.F1Score(average='macro', num_classes=NUM_CLASSES)
        self.accu_val_epoch = torchmetrics.Accuracy(average='macro', num_classes=NUM_CLASSES)
        self.F1_val_epoch = torchmetrics.F1Score(average='macro', num_classes=NUM_CLASSES)
        self.backbone = models.resnet50(weights="IMAGENET1K_V2")
        self.classifier = nn.Linear(1000, NUM_CLASSES)
        
    def forward(self,x):
        x = self.backbone(x)
        x = self.classifier(x)
        return(x)
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        
        self.accuracy(y_pred, y)
        self.F1(y_pred, y)
        loss = self.loss(y_pred,y)
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.log("train_acc", self.accuracy, prog_bar=True, logger=True)
        self.log('train_F1', self.F1, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = self.loss(y_pred,y)
        self.log("val_loss" ,loss )
        return {"loss": loss, "predictions": y_pred.detach(), "labels": y}
    
    def validation_epoch_end(self, voutputs):
        labels = []
        predictions = []
        tloss = []
        for output in voutputs:
            tloss.append(output["loss"])
            for out_labels in output["labels"].detach().cuda():
                labels.append(out_labels)
            for out_predictions in output["predictions"].detach().cuda():
                predictions.append(out_predictions)
                
        labels = torch.stack(labels)
        predictions = torch.stack(predictions)
        # print(f'Val shape {predictions.shape}')
        tloss = torch.stack(tloss)
        vloss = torch.mean(tloss)
        
        self.accu_val_epoch(predictions, labels)
        self.F1_val_epoch(predictions, labels)
        
        self.log('val_loss_epoch', vloss, prog_bar=True, logger=True)
        self.log('val_accu_epoch', self.accu_val_epoch, prog_bar=True, logger=True)
        self.log('val_F1_epoch', self.F1_val_epoch, prog_bar=True, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters() , 1e-3)
        sch = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma=0.1)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": sch,
                "monitor": "val_loss",
            },
        }

In [15]:
progress_callback = TQDMProgressBar(5)
checkpoint_callback = ModelCheckpoint(dirpath="checkpoints/",
                filename="best-checkpoint", save_top_k=1,
                monitor = "val_F1_epoch", mode = "max")
early_stoping_callback = EarlyStopping(monitor='val_F1_epoch', patience=2)
logger = TensorBoardLogger("light_logs/", name="dermalog")

In [16]:
model = ImageClassifier()

In [17]:
trainer = pl.Trainer(max_epochs=3, gpus = 1, 
        callbacks = [progress_callback, checkpoint_callback, early_stoping_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                 | Params
--------------------------------------------------------
0 | loss           | MultiLabelMarginLoss | 0     
1 | accuracy       | Accuracy             | 0     
2 | F1             | F1Score              | 0     
3 | accu_val_epoch | Accuracy             | 0     
4 | F1_val_epoch   | F1Score              | 0     
5 | backbone       | ResNet               | 25.6 M
6 | classifier     | Linear               | 23.0 K
--------------------------------------------------------
25.6 M    Trainable params
0         Non-trainable params
25.6 M    Total params
102.320   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

In [10]:
lodel = ImageClassifier.load_from_checkpoint('checkpoints/best-checkpoint.ckpt')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/15148/OneDrive - Vanier College/Documents/GitHub/NovaSci/checkpoints/best-checkpoint.ckpt'

In [ ]:
traineroo = pl.Trainer(max_epochs=9, gpus = 1, 
        callbacks = [progress_callback, checkpoint_callback, early_stoping_callback])

In [ ]:
traineroo.fit(lodel, data_module)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/